# Classification

Let's get learning, yaaaaaaaaaaaaay

There are many many different typese of classifications. Essentialy classification, pridicts what class a certian objecy maybe in. It works with the probability of the item being in each class. In this example we have flowers and we will use thier properties to find which species they are in.

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output

from tensorflow import feature_column as fc
import tensorflow as tf

This specific dataset seperates flowers into 3 different classes of species.
- Setosa
- Versicolor
- Virginica

The information about each flower is the following.
- sepal length
- sepal width
- petal length
- petal width

In [14]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']
# Lets define some constants to help us later on

train_path = tf.keras.utils.get_file(
    "iris_training.csv", "Dataset/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "Dataset/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
# Here we use keras (a module inside of TensorFlow) to grab our datasets and read them into a pandas dataframe

In [15]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [16]:
train_y = train.pop('Species')
test_y = test.pop('Species')
train.head() # the species column is now gone

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


We have 120 entries with 4 shapes.

In [17]:
train.shape  # we have 120 entires with 4 features

(120, 4)

Imput Function:
- This input function is very different, need to learn it more.

In [18]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

Yay feature columns :/
- Easy to make feature columns because we dont have any catagorical data.

In [ ]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

Creating the Model. For classifiers you can use many different models. Like:
- DNNClassifier (Deep Neural Network)
- LinearClassifier
- BoostedTreesClassifier
- DNNLinearCombinedClassifier \
And many other options. But we will use a DNNClassifier for this lab. DNNClassifier is \
a classifier for TensorFlow that will create a neural network with layers. It stand for \
Deep Neural Network. A linear classifier is a classifier that uses a linear combination \
and gives
the probability of an object being a certain class.

In [ ]:
# Building the model with 2 hidden layers with 30 and 10 hidden nodes each

classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively
    hidden_units=[30, 10],
    # The model must choose between 3 classes
    n_classes=3)

What we have done is created a DNN with 2 layers, with 30 and 10 neurons respectivey \
This is an arbitrary number and numerous tests are determined to choose the best value.

## Now it is time to Train the model

In [22]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=50000)
# We include a lambda to avoid creating an inner function previously

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpi7hlcwzh/model.ckpt-5000
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 5000...
INFO:tensorflow:Saving checkpoints for 5000 into /tmp/tmpi7hlcwzh/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 5000...
INFO:tensorflow:loss = 0.5151182, step = 5000
INFO:tensorflow:global_step/sec: 1111.35
INFO:tensorflow:loss = 0.5112151, step = 5100 (0.091 sec)
INFO:tensorflow:global_step/sec: 1499.72
INFO:tensorflow:loss = 0.49148473, step = 5200 (0.067 sec)
INFO:tensorflow:global_step/sec: 1711.83
INFO:tensorflow:loss = 0.4998353, step = 5300 (0.058 sec)
INFO:tensorflow:global_step/sec: 1721.88
INFO:tensorflow:loss = 0.47856146,

In [32]:
eval_result = classifier.evaluate(input_fn=lambda: input_fn(test, test_y, training=False))

clear_output()

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))
# We include a lambda to avoid creating an inner function previously


Test set accuracy: 0.967



Trained the data with 50,000 STEPS!!!, It gave very high accuracy

## Predictions:

This script allows you to rest the data with your specific values and check what the predicted class of that flower is

In [36]:
def input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print("Please type numeric values as prompted.")
for feature in features:
  valid = True
  while valid: 
    val = input(feature + ": ")
    if not val.isdigit(): valid = False

  predict[feature] = [float(val)]

predictions = classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(
        SPECIES[class_id], 100 * probability))


Please type numeric values as prompted.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpi7hlcwzh/model.ckpt-55000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Setosa" (98.4%)
